<a href="https://colab.research.google.com/github/erasmomendez/03MIARAlgoritmos/blob/main/Proyecto_Problema_de_sesiones_de_doblaje_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Erasmo Oscar Mendez Chavez  <br>
Url: <br>
Problema:
> 1. Sesiones de doblaje <br>


Descripción del problema: Organizar sesiones de doblaje

Proporciona el algoritmo basado en métodos heurísticos que soluciona el siguiente problema: Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los datos son:
Número de actores: 10
Número de tomas : 30
Actores/Tomas : https://bit.ly/36D8Iu.



(*) La respuesta es obligatoria







(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

In [ ]:
# a) Para calcular la cantidad de posibilidades sin tener en cuenta las restricciones consideramos que:
#    Cada toma mantiene sus actores que estaban predefinidos en la tabla inicial de tomas.
#    A partir de la tabla de 30 tomas obtenemos todas las posibles ordenaciones sin restricciones de este conjunto de 30 tomas.
#    Por tanto la cantidad de posibilidades sin restricciones es: n!
#    El resultado para n = 30 es: 30! = 2,6525285981219105863630848e+32

# b) Para calcular la cantidad de posibilidades teniendo en cuenta las restricciones consideramos que:
#    Es una combinación de 30 tomas que pueden ser escogidas de 6 tomas en 6 tomas (tomas por día),
#    Donde las posibles muestras sin orden de las 6 tomas distintas que se pueden extraer del totas de 30 tomas.
#    Por tanto la cantidad de posibilidades con restricciones se calcula con la fórmula: n! / (r!*(n-r)!)
#    Para la pregunta n = 30 y r = 6.  Aplicando la formula: 30!/(6!*(30-6)!)
#    El resultado es: 593,775 posibilidades


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

In [ ]:
# La estructura de datos que finalmente utilicé fue una matriz de enteros de filas de tomas(30) x columnas de actores(10).
# Inicialmente, intenté trabajar con un dataframe pero se complico la programación en el sentido que
# para los diversas asignaciones y cálculos que deseaba realizar en simultaneo, me encontraba con conflictos irresolubles entre strings y enteros;
# además para resolver estos problemas tenñia que convertir dataframes a vectores y arreglos (ida y vuelta) por lo que descarte el uso de dataframe.
# Para este tipo de problema el uso de vectores y array facilita las asignaciones, cálculos y sumatorias.


Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

In [ ]:
# 1) La función objetivo es obtener el mejor plan con el menor número posible de desplazamientos diario de actores,
#    dado que el actor cobra por día trasladado indiferente a las tomas que realice, es decir al menor coste posible .
# 2) Es un problema de minimización porque la función objetivo tiene como meta minimizar el coste total del plan

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [ ]:
import random
import numpy as np
import pandas as pd

In [ ]:
# Función objetivo: calcular el costo total de la asignacion de actores a un grupo de seis tomas

def calcular_costo(planificacion, costo_desplazamiento, num_actores, num_tomas):
    costo_total = 0
    acum_6_planif = np.zeros((num_actores), dtype=int)

#   identifica y acumula el numero de actores para el grupo de seis tomas
    i = 0
    for j in range(0,num_tomas):
        for k in range(0,num_actores):
            if i > 0:
                acum_6_planif[k] += planificacion[j-1,k].copy()
                if acum_6_planif[k] > 1:
                    acum_6_planif[k] = 1
            if i == 5:
                acum_6_planif[k] += planificacion[j,k].copy()
                if acum_6_planif[k] > 1:
                    acum_6_planif[k] = 1

        i += 1

#       Calcula el coste del numero de actores de seis tomas
        if i> 5:
            actores_6tomas = 0
            for k in range(0,num_actores):
                actores_6tomas += acum_6_planif[k]
            costo_total += (actores_6tomas * costo_desplazamiento)
            i = 0
            acum_6_planif = np.zeros((num_actores), dtype=int)

    return costo_total

In [ ]:
# Generación iterativa de varios planes de asignación de actores

def generar_plan_heuristica(matriz_actores, num_tomas, num_actores, costo_desplazamiento, num_iteraciones):
    mejor_planificacion = None
    mejor_costo = float('inf')

    # Generar una planificación aleatoria inicial
    for _ in range(num_iteraciones):
        planificacion = []
        tomas_disponibles = list(range(num_tomas))
        random.shuffle(tomas_disponibles)

        while len(tomas_disponibles) > 0:
            grupo = []
            for _ in range(min(6, len(tomas_disponibles))):
                toma = tomas_disponibles.pop()
                grupo.append(matriz_actores[toma])
            planificacion.append(grupo)
        planif_arreglo = np.array(planificacion).reshape(num_tomas, num_actores)

        # Calcular el costo de la planificación inicial
        costo_actual = calcular_costo(planif_arreglo, costo_desplazamiento, num_actores, num_tomas)

        # Si el costo del nuevo plan es menor, actualizar la planificación
        if costo_actual < mejor_costo:
            mejor_planificacion = planif_arreglo.copy()
            mejor_costo = costo_actual


    # Búsqueda local: intentar mejorar la planificación inicial
        nueva_planif = mejor_planificacion.copy()

        for j in range(0,num_tomas):
            for k in range(j+1,num_tomas):
                # Intercambiar la toma actual por la toma previa de la lista de tomas
                nuevo_grupo1 = nueva_planif[j,:].copy()
                nuevo_grupo2 = nueva_planif[k,:].copy()
                nueva_planif[k,:] = nuevo_grupo1.copy()
                nueva_planif[j,:] = nuevo_grupo2.copy()

                # Calcular el costo del nuevo plan
                nuevo_costo = calcular_costo(nueva_planif, costo_desplazamiento, num_actores, num_tomas)

                # Si el costo del nuevo plan es menor, actualizar la planificación
                if nuevo_costo < mejor_costo:
                    mejor_planificacion = nueva_planif.copy()
                    mejor_costo = nuevo_costo

    return mejor_planificacion, mejor_costo

In [ ]:
# Datos de entrada
num_iteraciones = 100
limite_tomas_dia = 6
num_actores = 10
num_tomas = 30
costo_desplazamiento = 1
matriz_actores = np.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                           [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                           [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
                           [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
                           [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
                           [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                           [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                           [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
                           [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
                           [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
                           [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
                           [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
                           [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                           [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                           [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
                           [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
                           [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                           [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                           [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
                           [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                           [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                           [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
                           [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
                           [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                           [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                           [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
                           [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

# Generar planificación heurística
plan_optimo, costo_total_optimo = generar_plan_heuristica(matriz_actores, num_tomas, num_actores, costo_desplazamiento, num_iteraciones)

# Presentar la mejor tabla de planificación de grupo de actores por toma y por día. Se incluye el más óptimo costo.
dia = 1
for i, grupo in enumerate(plan_optimo):
    if (round(i//limite_tomas_dia))>=1:
        dia = round(i//limite_tomas_dia) + 1
    print("Toma:", i+1, "Dia:", dia, "Grupo de actores:", grupo)
print("Costo total óptimo:", costo_total_optimo)

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

El código descrito como respuesta de la pregunta "Diseña un algoritmo para resolver el problema por fuerza bruta" tiene dos algoritmos:

1) Algoritmo de busqueda aleatoria. Que corresponde como respuesta a la pregunta "Diseña un algoritmo para RESOLVER por fuerza bruta"

2) Algoritmo de busqueda local. Este algoritmo responde a esta pregunta "Diseña un algoritmo que MEJORE la complejidad del algortimo por fuerza bruta". En este algortimo generaba de manera iterativa combiando un plan por busqueda aleatoria para luego revisar toda la lista por busqueda local.

EL resultado me permitio mejorar los valores obtenidos en la busqueda aleatoria
Esta es la razon por que los dos algoritmos estan unidos en un solo codigo. Entiendo tambien que otra opcion era haber generado en dos funciones



(*)Calcula la complejidad del algoritmo

Respuesta

DATOS DE PARTIDA
n = número de tomas = 30
m = número de actores = 10
L = número de iteraciones = 250
Supuesto: operaciones numpy de array, asignación, reshape, random y zeros son considerados como una operación elemental

CALCULOS

a) Función calcular costo

2 operaciones elementales

n *(m * 4) operaciones elementales

(n/5)* (m operaciones elementales)

Total = 2 + (n * m * 4) + ((n/5) * m) = 2 + (21 * m * n)/5

Total redondeado = (21 * m * n)/5


b) Generacion del plan aleatorio inicial

7 asignaciones

n *((n/5) * 2) asignaciones o condicionales

Función calcular costo = (21 * m * n)/5

Total = 7 + 2 (n( n/5)) + (21 * m * n)/5 = 2 (n( n/5)) + (21 * m * n)/5 Omitimos el 7


c) Generacion del plan por búsqueda local

1 asignación

n ((n-1)(4 + Función calcular costo))

Total = 1 + n ((n-1)(4 + (21 * mn)/5)) = (20n^2 + 21nn^3 – 20n – 21m*n^2)/5 Omitimos el 1


.


ALGORITMO PRINCIPAL: Generacion total del plan heurístico –

2 asignaciones

L * (Generación del plan aleatorio inicial + Generación del plan por búsqueda local)

Total = L * (((22n^2) + (21mn^3) - (21mn^2) + (42 *mn) – (20*n))/5)

Total = 250 * (((2230^2) + (213010^3) - (213010^2) + (42 *3010) – (20*10))/5)

281’960,000 de OPERACIONES ELEMENTALES

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta